In [ ]:
!pip install transformers==4.37.2

In [ ]:
!pip install accelerate bitsandbytes datasets peft einops torch

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# **در این قسمت کتابخانه های مورد نیاز را وارد میکنیم**

In [6]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os
import pandas as pd


# **در این قسمت توکنایزر مدلی که میخوایم فاین توین کنیم را از هاگینگ فیس دانلود میکنیم و آن را کانفیگ میکنیم. **

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [ ]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (dense): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear4bit(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernor

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "dense"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 6,291,456 || all params: 1,424,562,176 || trainable%: 0.4416413762764399


In [ ]:
print(model)

In [ ]:
def tokenize(sample):
    model_inps =  tokenizer(sample["query"], padding=True, truncation=True, max_length=512)
    return model_inps

# **دیتاست مورد نظر را از کتابخانه سایکت لیرن دانلود میکنیم و آن را به دو قسمت تست و ترین تقسیم میکنیم**

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(newsgroups_train["data"], newsgroups_train["target"], test_size=0.2, random_state=42)

In [ ]:
# Convert training data to a DataFrame
train_df = pd.DataFrame({"text": X_train, "target": y_train})

# Convert testing data to a DataFrame
test_df = pd.DataFrame({"text": X_test, "target": y_test})

In [ ]:
target_map = {i: newsgroups_train.target_names[i] for i in range(len(newsgroups_train.target_names))}

# Map target values to class names in the training and testing DataFrames
train_df['target'] = train_df['target'].map(target_map)
test_df['target'] = test_df['target'].map(target_map)

In [ ]:
train_df.head()

,text,target
0,From: Rupin.Dang@dartmouth.edu (Rupin Dang)\nS...,misc.forsale
1,From: simon@giaeb.cc.monash.edu.au\nSubject: S...,soc.religion.christian
2,From: sun075!Gerry.Palo@uunet.uu.net (Gerry Pa...,soc.religion.christian
3,From: downs@helios.nevada.edu (Lamont Downs)\n...,comp.os.ms-windows.misc
4,From: U52885@uicvm.uic.edu\nSubject: Re: Gatew...,comp.sys.ibm.pc.hardware


# **دیتا را با توجه به نیاز های مدل اماده میکنیم**

In [ ]:
train_df["query"] = train_df[["text", "target"]].apply(lambda x: "question: " + x["text"] + " answer: " + x["target"], axis=1)
data = Dataset.from_pandas(train_df)
tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)
tokenized_data

Tokenizing data:   0%|          | 0/9051 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 9051
})

# **پارامتر های اموزش را ست میکنیم و مدل را آموزش میدهیم**

In [ ]:
training_arguments = TrainingArguments(
        output_dir="phi-1_5-finetuned",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=1000,
        num_train_epochs=1,
        push_to_hub=True,
        optim="paged_adamw_32bit",
    )

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()
trainer.push_to_hub()

Step,Training Loss
100,3.203100
200,3.015600
300,3.003700
400,2.936700
500,2.928200
600,2.899600
700,2.902700
800,2.848400
900,2.888800
1000,2.883600


CommitInfo(commit_url='https://huggingface.co/mahdihassanzadeh/phi-1_5-finetuned/commit/15bc2099dff9548b6c181dfa942d972ee54c69c6', commit_message='End of training', commit_description='', oid='15bc2099dff9548b6c181dfa942d972ee54c69c6', pr_url=None, pr_revision=None, pr_num=None)

# **مدل را در سایت هاگینگ فیس  آپلود میکنیم.**

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True, torch_dtype=torch.float32)
peft_model = PeftModel.from_pretrained(model, "mahdihassanzadeh/phi-1_5-finetuned", from_transformers=True)
model = peft_model.merge_and_unload()
model

adapter_config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/25.2M [00:00<?, ?B/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2048,), e

In [ ]:
model.push_to_hub("mahdihassanzadeh/phi-1_5-finetuned")

README.md:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/688M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mahdihassanzadeh/phi-1_5-finetuned/commit/2bbd22b7ab248664b670c1aec45d8aad749438e5', commit_message='Upload PhiForCausalLM', commit_description='', oid='2bbd22b7ab248664b670c1aec45d8aad749438e5', pr_url=None, pr_revision=None, pr_num=None)